In [1]:
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Grouper
from pandas import DataFrame
import csv
import pickle
import math

/home/jonathan/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("/home/jonathan/tesis/Data_Temperatura_Lluvia/2020_Accidentalidad.csv",sep=';')#,index_col=['fecha'], parse_dates=True)
DataAcc = df.drop(columns=['localizacion','numero','estado_meteorológico','tipo_vehiculo','tipo_persona','rango_edad','sexo','cod_lesividad','lesividad','coordenada_x_utm','coordenada_y_utm','positiva_alcohol','positiva_droga'])
DataAcc['ds'] = DataAcc['fecha'] + '/' + DataAcc['hora']
DataAcc['ds'] = pd.to_datetime(DataAcc['ds'], format='%d/%m/%Y/%H:%M:%S')
DataAcc = DataAcc.drop(columns=['fecha','hora'])
DataAcc = DataAcc.set_index('ds')
DataAcc = DataAcc.sort_index()
DataAcc

,num_expediente,cod_distrito,distrito,tipo_accidente
ds,,,,
2020-01-01 01:00:00,2020S000014,3.0,RETIRO,Colisión fronto-lateral
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000014,3.0,RETIRO,Colisión fronto-lateral
...,...,...,...,...
2020-12-31 21:55:00,2020S019513,19.0,VICÁLVARO,Atropello a persona
2020-12-31 22:00:00,2020S019516,12.0,USERA,Choque contra obstáculo fijo
2020-12-31 22:25:00,2020S019517,13.0,PUENTE DE VALLECAS,Choque contra obstáculo fijo


In [3]:
# df = pd.read_csv("/home/jonathan/tesis/Data_Temperatura_Lluvia/2020_Accidentalidad.csv",sep=';')#,index_col=['fecha'], parse_dates=True)
# DataAcc = df.drop(columns=['localizacion','numero','estado_meteorológico','tipo_vehiculo','tipo_persona','rango_edad','sexo','cod_lesividad','lesividad','coordenada_x_utm','coordenada_y_utm','positiva_alcohol','positiva_droga'])
# DataAcc['fecha'] = pd.to_datetime(DataAcc['fecha'], format='%d/%m/%Y')
# DataAcc = DataAcc.set_index('fecha')
# DataAcc = DataAcc.sort_index()
# DataAcc

In [4]:
DataAcc1 = DataAcc.loc[((DataAcc['cod_distrito'] == 16) | (DataAcc['cod_distrito'] == 8)) # Los números del distrito cambian
                       & ((DataAcc['tipo_accidente'] != 'Caída') & (DataAcc['tipo_accidente'] != 'Otro'))]
DataAcc1['Accidente'] = True
DataAcc1 = DataAcc1.drop_duplicates(subset = ['num_expediente'])
# DataAcc1 = DataAcc1.loc[:,['hora','Accidente']]

/tmp/ipykernel_14524/484863067.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataAcc1['Accidente'] = True


In [6]:
DataAcc1 = DataAcc1.reset_index()

In [7]:
def ModTime(ds):
    date = pd.to_datetime(ds)
    hora = date.hour
    minuto = date.minute
    if (hora != 23) & (minuto >= 30):   # Revisar si esta bien esta condición
        date = date.replace(hour=hora+1,minute=0)
    else:
        date = date.replace(minute=0)
    return (date)
DataAcc1['ds'] = DataAcc1['ds'].apply(ModTime)

In [8]:
DataAcc1

,ds,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente
0,2020-01-01 01:00:00,2020S000002,16.0,HORTALEZA,Choque contra obstáculo fijo,True
1,2020-01-01 10:00:00,2020S000142,8.0,FUENCARRAL-EL PARDO,Choque contra obstáculo fijo,True
2,2020-01-01 12:00:00,2020S000206,8.0,FUENCARRAL-EL PARDO,Choque contra obstáculo fijo,True
3,2020-01-01 14:00:00,2020S000424,16.0,HORTALEZA,Choque contra obstáculo fijo,True
4,2020-01-01 16:00:00,2020S000061,16.0,HORTALEZA,Colisión fronto-lateral,True
...,...,...,...,...,...,...
1313,2020-12-31 17:00:00,2020S019601,16.0,HORTALEZA,Colisión fronto-lateral,True
1314,2020-12-31 19:00:00,2020S019598,16.0,HORTALEZA,Choque contra obstáculo fijo,True
1315,2020-12-31 20:00:00,2020S019502,8.0,FUENCARRAL-EL PARDO,Atropello a persona,True
1316,2020-12-31 21:00:00,2020S019582,16.0,HORTALEZA,Colisión fronto-lateral,True


In [9]:
#############Prueba###############
# DataAccPrue2 = DataAcc1.loc[DataAcc1['tipo_accidente'] == 'Caída'] 
# DataAccPrue2.shape
# DataAccPrue2 = DataAcc1.loc[(DataAcc1['cod_distrito'] != 16) & (DataAcc1['cod_distrito'] != 8)] 
# DataAccPrue2.shape

(0, 6)